In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal, P_C):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W + k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W + k_hive)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W + k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W - P_C
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal, P_C):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal, P_C), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t 

In [ ]:
def residual(ps, ts, data, l):
    d = pd.DataFrame(data).groupby(data['Date'])
    model = []
    k = 0
    alpha  = 1
    r_c = 124 #1011 #ps['w1'].value
    r_h = 428 #6204 #ps['w2'].value
    h_ = [1]
    return_value = []
    for m,n in d:
        h = ps['h_'+str(k)].value #health factor per day
        theta_ideal = ps['theta_'+str(k)].value + 273.15 #theta_ideal per day
        t_max = len(n['Air Temp'])-1
        t = np.linspace(0,t_max, num = t_max+1)
        environment_temp = n['Air Temp'].values + 273.15 #hourly environment temperature -> kelvin
        solar_rad = n['Sol Rad'].values #hourly solar radiation -> kelvin
        if n['Treatment'].values[0] == 1:
            P_C = n['P'].values
        else:
            P_C = 0
        #k_W = n['k_W'].values
        #k_A_W = n['k_A_W'].values
        #k_hive = n['k_hive'].values
        #eta_W = n['eta_W'].values
        #eta_A_W = n['eta_A_W'].values
        #eta_B_W = n['eta_B_W'].values
        
        a = [1]*len(environment_temp)
        
        fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                            k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal, P_C)
        model = np.concatenate((model, fitted - 273.15))
        try:
            h_.append(abs(h - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - h))
        k = k+1
    return_value = np.concatenate((return_value, (model - data['Temp']).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    print(np.mean(abs((model - data['Temp']).ravel())))
    return return_value

In [ ]:
import datetime
data = pd.read_csv('C:\Bee Temp Data\C294_w_Env.csv')

try:
    for i in range(len(data)):
        current_date = (datetime.datetime.strptime(str(data['Date'].values[i]), "%m/%d/%Y")).date()
        current_time = datetime.datetime.strptime(str(data['Time'].values[i]), "%H:%M:%S").time()
        current_datetime = datetime.datetime.combine(current_date,current_time)
        data['DateTime'][i] = datetime.datetime.strptime(str(current_datetime), "%Y-%m-%d %H:%M:%S")
        data['Date'][i] = data['DateTime'][i].date()
        data['Time'][i] = data['DateTime'][i].time()
except:
    pass


d = pd.DataFrame(data).groupby([data['Date']])
k_W = 0.29 #0.9322636905520281 #[9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
eta_W = 0.0085*k_W #0.0033021413705307534*k_W #[0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
k_A_W = 0.29 #0.6699726312530309 #[7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = 0.0085*k_W #0.0021755653867763236*k_A_W #[0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
eta_B_W = 0.0085*k_W #0.0029*k_W #[0.0029*9.974510910778044]*len(data) #heat absorption coeff of bottom surface
k_hive = 0.8 #1.07 #0.84 #[1.5540477862285431]*len(data)
        
#data['k_W'] = k_W
#data['eta_W'] = eta_W
#data['k_A_W'] = k_A_W
#data['eta_A_W'] = eta_A_W
#data['eta_B_W'] = eta_B_W
#data['k_hive'] = k_hive

In [ ]:
h_max = 1
l = 96/h_max
params = Parameters()
a = 0
for i,j in d:
    params.add('h_'+str(a), value = 0.5, min = 0.1, max = h_max)
    try:
        params.add('theta_'+str(a), value = np.mean(j['Temp']), min = np.min(j['Temp']), max = np.min(j['Temp']))
    except:
        params.add('theta_'+str(a), value = np.mean(j['Temp']), min = np.min(j['Temp']) - 0.05, max = np.max(j['Temp']) + 0.05)
    a = a+1             
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)

result = minimize(residual, params, args=(t,data, l), method='leastsq',nan_policy='omit')
sig_min = data['Temp'].values + result.residual[:len(data['Temp'].values)].reshape(data['Temp'].shape)

In [ ]:
data['Recons_P'] = sig_min

h = np.array([])
theta = np.array([])
a = 0
for i in range(len(d)):
    h = np.concatenate((h, np.array([result.params['h_'+str(a)].value]*24)))
    theta = np.concatenate((theta, np.array([result.params['theta_'+str(a)]]*24)))
    a = a+1
data['h_P'] = h
data['theta_P'] = theta

In [ ]:
data.to_csv('C:\Bee Temp Data\C294_w_Env.csv')

In [ ]:
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
plt.plot(sig_min, label = 'Reconstructed Core Temp', color = 'red') 
plt.plot(data['theta_P'])
plt.xlabel('Samples per hour')
plt.ylabel('Temperature in celcius')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W + k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W+k_hive)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W+k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W 
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t  

In [ ]:
data = pd.read_csv('C:\Bee Temp Data\C260_w_Env_only_treat.csv')
#data = pd.read_csv('C:\Bee Temp Data\C260_w_Env.csv')
data.tail()

In [ ]:
def bee_wall_eq(y, t, env_temp, solar, k_W, k_hive, eta_W, hive_temp, P):
    wall_temp = y
    A_W = k_W*(-wall_temp + env_temp) + eta_W*solar + k_hive*(-wall_temp + hive_temp) - P
    dydt = A_W 
    return dydt

def run_bee_wall_eq(t, a, env_temp, solar, k_W, k_hive, eta_W, hive_temp, P):
    sol = scint.odeint(bee_wall_eq, a, t, args=(env_temp, solar, k_W, k_hive, eta_W, hive_temp, P), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t  

In [ ]:
h = data['h_P'].values #health factor per day
theta_ideal = data['theta_P'].values + 273.15 #theta_ideal per day
t_max = len(data['Air Temp'])-1
t = np.linspace(0,t_max, num = t_max+1)
environment_temp = data['Air Temp'].values + 273.15 #hourly environment temperature -> kelvin
solar_rad = data['Sol Rad'].values #hourly solar radiation -> kelvin
hive_temp = data['Temp'].values + 273.15
#k_W = data['k_W'].values
#k_A_W = data['k_A_W'].values
#k_hive = data['k_hive'].values
#eta_W = data['eta_W'].values
#eta_A_W = data['eta_A_W'].values
#eta_B_W = data['eta_B_W'].values
k_W = 0.29 #0.9322636905520281 #[9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
eta_W = 0.0017*0.29 #0.0033021413705307534*k_W #[0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
k_A_W = 0.29 #0.6699726312530309 #[7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = 0.0017*0.29 #0.0021755653867763236*k_A_W #[0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
eta_B_W = 0.0017*0.29 #0.0029*k_W #[0.0029*9.974510910778044]*len(data) #heat absorption coeff of bottom surface
k_hive = 0.41 #1.07 #0.84 #[1.5540477862285431]*len(data)
P = data['P'].values

r_c = 124#1011 #ps['w1'].value
r_h = 428#6204 #ps['w2'].value
        
a = [1]*len(environment_temp)
        
fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                    k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal)

fitted_wall = run_bee_wall_eq(t, a, environment_temp, solar_rad, k_W, k_hive, eta_W, hive_temp, P)

positive_change = data['P'] > 0

plt.figure(figsize = (10,4), dpi = 200)
xlabel = data['DateTime'].values
y = xlabel[range(0,len(xlabel),24)]
legend_properties = {'size': 10}
ax1 = plt.subplot(211)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax1.plot(data['Temp'], label = 'Recorded Core Temp /w Ice', color = 'orange')
ax1.plot(fitted -273.15, label = 'Reconstructed Core Temp w/o Ice', color = 'red') 
ax1.plot(data['Recons_P'], label = 'Reconstructed Core Temp /w Ice', color = 'blue')

ax1.fill_between(range(len(fitted)), fitted - 273.15, data['Recons_P'], where = positive_change, color='m', alpha=0.3, label='Reduced Core Temperature')
ax1.set_ylabel('Temperature($^0C$)')
ax1.legend(loc='lower right', prop=legend_properties, ncol = 2)
ax1.yaxis.set_tick_params(labelsize=12)
ax1.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date']),12))
#ax1.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date'].values),24))
ax1.set_xticklabels(y,rotation=90)
ax1.set_xticks([])

ax2 = plt.subplot(212)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax2.plot(data['Temp_Box'], label = 'Empty Box Temperature', color = 'orange')
ax2.plot(data['Recons_Peri'], label = 'Reconstructed Peri Temperature w/o ice', color = 'blue')
ax2.plot(data['Temp_Easy'], label = 'Peri Temp /w ice', color = 'red') 

ax2.fill_between(range(len(data['Temp_Box'])), data['Temp_Box'], data['Temp_Easy'], where = positive_change, color='m', alpha=0.3, label='Saved Bee Energy for Cooling')
ax2.set_xlabel('Date & Time')
ax2.set_ylabel('Temperature($^0C$)')
ax2.legend(loc='lower left', prop=legend_properties)
ax2.yaxis.set_tick_params(labelsize=12)
ax2.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax2.set_xticks(range(0,len(data['Date']),12))
ax2.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax2.set_xticks(range(0,len(data['Date'].values),24))
ax2.set_xticklabels(y,rotation=90)
plt.legend()
plt.show()

In [ ]:
#data = pd.read_csv('C:\Bee Temp Data\C260_w_Env.csv')
#data['Fitted'] = fitted
#data = data[data['Treatment'] == 1]

#print(len(data))
actual_peri = []

for i in range(len(data)):
    if data['Treatment'].values[i] == 1:
        actual_peri.append(data['Recons_Peri'].values[i])
    else:
        actual_peri.append(fitted_wall[i] - 273.15)

actual_bee_energy_heating = ((actual_peri - fitted + 273.15)*[actual_peri < fitted - 273.15])[0]
actual_bee_energy_cooling = ((actual_peri - fitted + 273.15)*[actual_peri > fitted - 273.15])[0]
actual_bee_energy = actual_bee_energy_heating + actual_bee_energy_cooling
cooled_bee_energy_heating = ((fitted_wall - data['Recons_P'].values - 273.15)*[fitted_wall- 273.15 < data['Recons_P'].values])[0]
cooled_bee_energy_cooling = ((fitted_wall - data['Recons_P'].values - 273.15)*[fitted_wall - 273.15> data['Recons_P'].values])[0]
cooled_bee_energy = cooled_bee_energy_heating + cooled_bee_energy_cooling

actual_heating_energy = (2*abs(actual_peri - fitted + 273.15)*[actual_peri < fitted - 273.15])[0]
actual_cooling_energy = (5*abs(actual_peri - fitted + 273.15)*[actual_peri > fitted - 273.15])[0]

total_actual_energy = actual_heating_energy + actual_cooling_energy

cooled_heating_energy = (2*abs(fitted_wall - data['Recons_P'].values - 273.15)*[fitted_wall- 273.15 < data['Recons_P'].values])[0]
cooled_cooling_energy = (5*abs(fitted_wall - data['Recons_P'].values - 273.15)*[fitted_wall - 273.15> data['Recons_P'].values])[0]

total_cooled_energy = cooled_heating_energy + cooled_cooling_energy

positive_change = (actual_bee_energy - cooled_bee_energy) > 0 

plt.figure(figsize = (12,8), dpi = 200)
xlabel = data['DateTime'].values
y = xlabel[range(0,len(xlabel),24)]
legend_properties = {'size': 10}
ax1 = plt.subplot(311)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
#ax1.plot(data['Temp'], label = 'Recorded Core Temp /w Ice', color = 'orange')
ax1.plot(fitted -273.15, '--', label = 'Reconstructed Core Temp w/o Ice', color = 'red', alpha = 0.7) 
ax1.plot(data['Recons_P'], '--', label = 'Reconstructed Core Temp /w Ice', color = 'blue', alpha = 0.7)

positive_change = actual_peri > fitted - 273.15
ax1.fill_between(range(len(fitted)), fitted - 273.15, data['Recons_P'], where = positive_change, color='m', alpha=0.5, label='Reduced Core Temperature for Cooling')

negative_change = actual_peri < fitted - 273.15
ax1.fill_between(range(len(fitted)), fitted - 273.15, data['Recons_P'], where = negative_change, color='olive', alpha=0.5, label='Increase Core Temperature for Heating')

ax1.set_ylabel('Temperature($^0C$)')
ax1.legend(loc='lower left', prop=legend_properties, ncol = 2)
ax1.yaxis.set_tick_params(labelsize=12)
ax1.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date']),12))
#ax1.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date'].values),24))
ax1.set_xticklabels(y,rotation=90)
ax1.set_xticks([])

ax2 = plt.subplot(312)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax2.plot(actual_peri - fitted + 273.15, '--', label = '(Peri Temp - Core Temp) w/o ice', color = 'Red', alpha = 0.7)
ax2.plot(data['Temp_Easy'] - data['Recons_P'], '--', label = '(Peri Temp - Core Temp) /w Ice', color = 'blue', alpha = 0.7)
#ax2.plot(data['Temp_Easy'], label = 'Peri Temp /w ice', color = 'red') 

positive_change = actual_peri > fitted - 273.15
ax2.fill_between(range(len(data['Temp_Box'])), actual_peri - fitted + 273.15, data['Temp_Easy'] - data['Recons_P'], where = positive_change, color='m', alpha=0.5, label= 'Saved Bee Energy for Cooling')

negative_change = actual_peri < fitted - 273.15
ax2.fill_between(range(len(data['Temp_Box'])), actual_peri - fitted + 273.15, data['Temp_Easy'] - data['Recons_P'], where = negative_change, color='olive', alpha=0.5, label= 'Extra Bee Energy for Heating')

no_change = actual_peri - fitted + 273.15 < data['Temp_Easy'] - data['Recons_P']
ax2.fill_between(range(len(data['Temp_Box'])), actual_peri - fitted + 273.15, data['Temp_Easy'] - data['Recons_P'], where = no_change, color='white')

ax2.set_ylabel('Temperature($^0C$)')
ax2.legend(loc='lower left', prop=legend_properties, ncol = 2)
ax2.yaxis.set_tick_params(labelsize=12)
ax2.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax2.set_xticks(range(0,len(data['Date']),12))
ax2.set_xticks(range(0,len(data['Date'].values),24))
ax2.set_xticklabels(y,rotation=90)
ax2.set_xticks([])

ax3 = plt.subplot(313)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax3.plot(total_actual_energy, '--', label = 'Honey consumption w/o ice', color = 'Red', alpha = 0.7)
ax3.plot(total_cooled_energy, '--', label = 'Honey consumption /w Ice', color = 'blue', alpha = 0.7)
#ax2.plot(data['Temp_Easy'], label = 'Peri Temp /w ice', color = 'red') 

positive_change = actual_peri > fitted - 273.15

ax3.fill_between(range(len(data['Temp_Box'])), total_actual_energy, total_cooled_energy, where = positive_change, color='m', alpha=0.5, label='Saved Bee Energy from Cooling')
negative_change = actual_peri < fitted - 273.15 
ax3.fill_between(range(len(data['Temp_Box'])), total_actual_energy, total_cooled_energy, where = negative_change, color='olive', alpha=0.5, label='Extra Bee Energy for Heating')

ax3.fill_between(range(len(data['Temp_Box'])), total_actual_energy, total_cooled_energy, where = no_change, color='white')

ax3.set_xlabel('Date & Time')
ax3.set_ylabel('Honey (g)')
ax3.legend(loc='upper left', prop=legend_properties, ncol = 2)
ax3.yaxis.set_tick_params(labelsize=12)
ax3.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax3.set_xticks(range(0,len(data['Date']),12))
ax3.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax3.set_xticks(range(0,len(data['Date'].values),24))
ax3.set_xticklabels(y,rotation=90)
#plt.legend()
plt.show()

In [ ]:
actual_peri = []

for i in range(len(data)):
    if data['Treatment'].values[i] == 1:
        actual_peri.append(data['Recons_Peri'].values[i])
    else:
        actual_peri.append(fitted_wall[i] - 273.15)

actual_bee_energy_heating = ((actual_peri - fitted + 273.15)*[actual_peri < fitted - 273.15])[0]
actual_bee_energy_cooling = ((actual_peri - fitted + 273.15)*[actual_peri > fitted - 273.15])[0]
actual_bee_energy = actual_bee_energy_heating + actual_bee_energy_cooling
cooled_bee_energy_heating = ((fitted_wall - 273.15 - data['Recons_P'].values)*[fitted_wall - 273.15 < data['Recons_P'].values])[0]
cooled_bee_energy_cooling = ((fitted_wall - 273.15 - data['Recons_P'].values)*[fitted_wall - 273.15 > data['Recons_P'].values])[0]
cooled_bee_energy = cooled_bee_energy_heating + cooled_bee_energy_cooling

actual_heating_energy = (2*abs(actual_peri - fitted + 273.15)*[actual_peri < fitted - 273.15])[0]
actual_cooling_energy = (5*abs(actual_peri - fitted + 273.15)*[actual_peri > fitted - 273.15])[0]

total_actual_energy = actual_heating_energy + actual_cooling_energy

#cooled_heating_energy = (2*abs(data['Temp_Easy'].values - data['Recons_P'].values)*[data['Temp_Easy'].values < data['Recons_P'].values])[0]
#cooled_cooling_energy = (5*abs(data['Temp_Easy'].values - data['Recons_P'].values)*[data['Temp_Easy'].values > data['Recons_P'].values])[0]

actual_peri_5 = [x * 5 for x in actual_peri]
actual_peri_6 = [x * 5 for x in actual_peri]
recons_P_6 = [x * 6 for x in data['Recons_P'].values]
cooled_heating_energy = (2*(abs(fitted_wall - 273.15 + actual_peri_5 - recons_P_6)/6)*[fitted_wall - 273.15 + actual_peri_5 < recons_P_6])[0]
cooled_cooling_energy = (5*(abs(fitted_wall - 273.15 + actual_peri_5 - recons_P_6)/6)*[fitted_wall - 273.15 + actual_peri_5 > recons_P_6])[0]

total_cooled_energy = cooled_heating_energy + cooled_cooling_energy

positive_change = (actual_bee_energy - cooled_bee_energy) > 0 

plt.figure(figsize = (12,8), dpi = 200)
xlabel = data['DateTime'].values
y = xlabel[range(0,len(xlabel),24)]
legend_properties = {'size': 10}
ax1 = plt.subplot(311)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
#ax1.plot(data['Temp'], label = 'Recorded Core Temp /w Ice', color = 'orange')
ax1.plot(fitted -273.15, '--', label = 'Reconstructed $\hat{\Theta}(t)$ without ice', color = 'red', alpha = 0.7) 
ax1.plot(data['Recons_P'], '--', label = 'Reconstructed $\hat{\Theta}_{adj}(t)$ with ice', color = 'blue', alpha = 0.7)

positive_change = actual_peri > fitted - 273.15
ax1.fill_between(range(len(fitted)), fitted - 273.15, data['Recons_P'], where = positive_change, color='m', alpha=0.5, label='Reduced Core Temperature due to Ice-pack (Cooling)')

negative_change = actual_peri < fitted - 273.15
ax1.fill_between(range(len(fitted)), fitted - 273.15, data['Recons_P'], where = negative_change, color='olive', alpha=0.5, label='Increased Core Temperature due to Residual Effect (Heating)')

ax1.set_ylabel('Temperature($^0C$)')
ax1.legend(loc='lower left', prop=legend_properties, ncol = 2)
ax1.yaxis.set_tick_params(labelsize=12)
ax1.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date']),12))
#ax1.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date'].values),24))
ax1.set_xticklabels(y,rotation=90)
ax1.set_xticks([])

ax2 = plt.subplot(312)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax2.plot(actual_peri - fitted + 273.15, '--', label = '$E_C$ without ice', color = 'Red', alpha = 0.7)
ax2.plot((fitted_wall - 273.15 +actual_peri_5 - recons_P_6)/6, '--', label = '$E_C$ with Ice', color = 'blue', alpha = 0.7)
#ax2.plot(data['Temp_Easy'], label = 'Peri Temp /w ice', color = 'red') 

positive_change = actual_peri > fitted - 273.15
ax2.fill_between(range(len(data['Temp_Box'])), actual_peri - fitted + 273.15, (fitted_wall - 273.15+actual_peri_5 - recons_P_6)/6, where = positive_change, color='m', alpha=0.5, label= 'Saved Bee Energy due to Ice-pack (Cooling)')

negative_change = actual_peri < fitted - 273.15
ax2.fill_between(range(len(data['Temp_Box'])), actual_peri - fitted + 273.15, (fitted_wall - 273.15+actual_peri_5 - recons_P_6)/6, where = negative_change, color='olive', alpha=0.5, label= 'Extra Bee Energy Needed due to Residual Effect (Heating)')

#no_change = actual_peri - fitted + 273.15 < data['Temp_Easy'] - data['Recons_P']
#ax2.fill_between(range(len(data['Temp_Box'])), actual_peri - fitted + 273.15, data['Temp_Easy'] - data['Recons_P'], where = no_change, color='white')

ax2.set_ylabel('Temperature($^0C$)')
ax2.legend(loc='lower left', prop=legend_properties, ncol = 2)
ax2.yaxis.set_tick_params(labelsize=12)
ax2.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax2.set_xticks(range(0,len(data['Date']),12))
ax2.set_xticks(range(0,len(data['Date'].values),24))
ax2.set_xticklabels(y,rotation=90)
ax2.set_xticks([])

ax3 = plt.subplot(313)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax3.plot(total_actual_energy, '--', label = 'Honey consumption without ice', color = 'Red', alpha = 0.7)
ax3.plot(total_cooled_energy, '--', label = 'Honey consumption with Ice', color = 'blue', alpha = 0.7)
#ax2.plot(data['Temp_Easy'], label = 'Peri Temp /w ice', color = 'red') 

positive_change = actual_peri > fitted - 273.15

ax3.fill_between(range(len(data['Temp_Box'])), total_actual_energy, total_cooled_energy, where = positive_change, color='m', alpha=0.5, label='Saved Honey due to Ice-pack (Cooling)')
negative_change = actual_peri < fitted - 273.15 
ax3.fill_between(range(len(data['Temp_Box'])), total_actual_energy, total_cooled_energy, where = negative_change, color='olive', alpha=0.5, label='Extra Honey Needed due to Residual Effect Bee (Heating)')

#ax3.fill_between(range(len(data['Temp_Box'])), total_actual_energy, total_cooled_energy, where = no_change, color='white')

ax3.set_xlabel('Date & Time')
ax3.set_ylabel('Honey (g)')
ax3.legend(loc='upper left', prop=legend_properties, ncol = 2)
ax3.yaxis.set_tick_params(labelsize=12)
ax3.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax3.set_xticks(range(0,len(data['Date']),12))
ax3.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax3.set_xticks(range(0,len(data['Date'].values),24))
ax3.set_xticklabels(y,rotation=90)
#plt.legend()
plt.show()

In [ ]:
sum(- data['Recons_P_less_1_84'] + fitted - 273.15)

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal, k_c, P_C):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W + k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W+k_hive+k_c)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W+k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W - P_C
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal, k_c, P_C):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal, k_c, P_C), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t  

In [ ]:
def residual(ps, ts, data, l):
    d = pd.DataFrame(data).groupby(data['Date'])
    model = []
    k = 0
    alpha  = 1
    r_c = 1011 #ps['w1'].value
    r_h = 6204 #ps['w2'].value
    h_ = [1]
    return_value = []
    for m,n in d:
        h = ps['h_'+str(k)].value #health factor per day
        theta_ideal = ps['theta_'+str(k)].value + 273.15 #theta_ideal per day
        t_max = len(n['Air Temp'])-1
        t = np.linspace(0,t_max, num = t_max+1)
        environment_temp = n['Air Temp'].values + 273.15 #hourly environment temperature -> kelvin
        solar_rad = n['Sol Rad'].values #hourly solar radiation -> kelvin
        P_C = n['Ice_Pow'].values
        k_W = n['k_W'].values
        k_A_W = n['k_A_W'].values
        k_hive = n['k_hive'].values
        eta_W = n['eta_W'].values
        eta_A_W = n['eta_A_W'].values
        eta_B_W = n['eta_B_W'].values
        k_c = n['k_C'].values
        
        a = [1]*len(environment_temp)
        
        fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                            k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal, k_c, P_C)
        model = np.concatenate((model, fitted - 273.15))
        try:
            h_.append(abs(h - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - h))
        k = k+1
    return_value = np.concatenate((return_value, (model - data['Temp']).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    print(np.mean(abs((model - data['Temp']).ravel())))
    return return_value

In [ ]:
import datetime
data = pd.read_csv('C:\Bee Temp Data\C260_w_Env.csv')

try:
    for i in range(len(data)):
        current_date = (datetime.datetime.strptime(str(data['Date'].values[i]), "%m/%d/%Y")).date()
        current_time = datetime.datetime.strptime(str(data['Time'].values[i]), "%H:%M:%S").time()
        current_datetime = datetime.datetime.combine(current_date,current_time)
        data['DateTime'][i] = datetime.datetime.strptime(str(current_datetime), "%Y-%m-%d %H:%M:%S")
        data['Date'][i] = data['DateTime'][i].date()
        data['Time'][i] = data['DateTime'][i].time()
except:
    pass


d = pd.DataFrame(data).groupby([data['Date']])
k_W = [9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
eta_W = [0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
k_A_W = [7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = [0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
eta_B_W = [0.0029*9.974510910778044]*len(data) #heat absorption coeff of bottom surface
k_hive = [1.5540477862285431]*len(data)
k_c = []

for i in range(len(data)):
    if data['Ice_Pow'][i] != 0:
        k_c.append(10)
    else:
        k_c.append(0)
        
data['k_W'] = k_W
data['eta_W'] = eta_W
data['k_A_W'] = k_A_W
data['eta_A_W'] = eta_A_W
data['eta_B_W'] = eta_B_W
data['k_hive'] = k_hive
data['k_C'] = k_c

In [ ]:
data.head()

In [ ]:
h_max = 1
l = 96/h_max
params = Parameters()
a = 0
for i,j in d:
    params.add('h_'+str(a), value = 0.5, min = 0.1, max = h_max)
    try:
        params.add('theta_'+str(a), value = np.mean(j['Temp']), min = np.min(j['Temp']), max = np.max(j['Temp']))
    except:
        params.add('theta_'+str(a), value = np.mean(j['Temp']), min = np.min(j['Temp']) - 0.05, max = np.max(j['Temp']) + 0.05)
    a = a+1             
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)

result = minimize(residual, params, args=(t,data, l), method='leastsq',nan_policy='omit')
sig_min = data['Temp'].values + result.residual[:len(data['Temp'].values)].reshape(data['Temp'].shape)

In [ ]:
data['Recons_P_C'] = sig_min

h = np.array([])
theta = np.array([])
a = 0
for i in range(len(d)):
    h = np.concatenate((h, np.array([result.params['h_'+str(a)].value]*24)))
    theta = np.concatenate((theta, np.array([result.params['theta_'+str(a)]]*24)))
    a = a+1
data['h_P_C'] = h
data['theta_P_C'] = theta

In [ ]:
data.to_csv('C:\Bee Temp Data\C260_w_Env.csv')

In [ ]:
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
plt.plot(sig_min, label = 'Reconstructed Core Temp', color = 'red') 
plt.plot(data['theta_P_C'])
plt.xlabel('Samples per hour')
plt.ylabel('Temperature in celcius')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W + k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W+k_hive)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W+k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W 
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t  

In [ ]:
data = pd.read_csv('C:\Bee Temp Data\C260_w_Env_only_treat.csv')
data.tail()

In [ ]:
h = data['h_P_C_100'].values #health factor per day
theta_ideal = data['theta_P_C_100'].values + 273.15 #theta_ideal per day
t_max = len(data['Air Temp'])-1
t = np.linspace(0,t_max, num = t_max+1)
environment_temp = data['Air Temp'].values + 273.15 #hourly environment temperature -> kelvin
solar_rad = data['Sol Rad'].values #hourly solar radiation -> kelvin
k_W = data['k_W'].values
k_A_W = data['k_A_W'].values
k_hive = data['k_hive'].values
eta_W = data['eta_W'].values
eta_A_W = data['eta_A_W'].values
eta_B_W = data['eta_B_W'].values
r_c = 1011 #ps['w1'].value
r_h = 6204 #ps['w2'].value
        
a = [1]*len(environment_temp)
        
fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                    k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal)



positive_change = fitted - 273.15 > data['Recons_P_C_100']

plt.figure(figsize = (10,4), dpi = 200)
xlabel = data['DateTime'].values
y = xlabel[range(0,len(xlabel),24)]
legend_properties = {'size': 12,'weight':'bold'}
ax1 = plt.subplot(111)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax1.plot(data['Temp']+273.15, label = 'Recorded Core Temp /w Ice', color = 'orange')
ax1.plot(fitted, label = 'Reconstructed Core Temp w/o Ice', color = 'red') 
ax1.plot(data['Recons_P_C_100']+273.15, label = 'Reconstructed Core Temp /w Ice', color = 'blue')

ax1.fill_between(range(len(fitted)), fitted, data['Recons_P_C_100']+273.15, where = positive_change, color='m', alpha=0.3, label='Saved Bee Energy')
ax1.set_xlabel('Date & Time')
ax1.set_ylabel('Temperature (K)')
ax1.legend(loc='lower left', prop=legend_properties)
ax1.yaxis.set_tick_params(labelsize=12)
ax1.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date']),12))
ax1.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date'].values),24))
ax1.set_xticklabels(y,rotation=90)
plt.legend()
plt.show()

In [ ]:
data['Recons_no_ice'] = fitted - 273.15
data.to_csv('C:\Bee Temp Data\C268_w_Env_only_treat.csv')

In [ ]:
sum((fitted-273.15 - data['Recons_P_C'])[fitted-273.15 - data['Recons_P_C'] > 0])

In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W + k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W+k_hive)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W+k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t  

In [ ]:
def residual(ps, ts, data, l):
    d = pd.DataFrame(data).groupby(data['Date'])
    model = []
    k = 0
    alpha  = 1
    r_c = 1011 #ps['w1'].value
    r_h = 6204 #ps['w2'].value
    h_ = [1]
    return_value = []
    for m,n in d:
        h = ps['h_'+str(k)].value #health factor per day
        theta_ideal = ps['theta_'+str(k)].value + 273.15 #theta_ideal per day
        t_max = len(n['Air Temp'])-1
        t = np.linspace(0,t_max, num = t_max+1)
        environment_temp = n['Air Temp'].values + 273.15 #hourly environment temperature -> kelvin
        solar_rad = n['Sol Rad'].values #hourly solar radiation -> kelvin
        P_C = n['Ice_Pow'].values
        k_W = n['k_W'].values
        k_A_W = n['k_A_W'].values
        k_hive = n['k_hive'].values
        eta_W = n['eta_W'].values
        eta_A_W = n['eta_A_W'].values
        eta_B_W = n['eta_B_W'].values
        
        a = [1]*len(environment_temp)
        
        fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                            k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal)
        model = np.concatenate((model, fitted - 273.15))
        try:
            h_.append(abs(h - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - h))
        k = k+1
    return_value = np.concatenate((return_value, (model - data['Temp']).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    print(np.mean(abs((model - data['Temp']).ravel())))
    return return_value

In [ ]:
import datetime
data = pd.read_csv('C:\Bee Temp Data\C260_w_Env.csv')

try:
    for i in range(len(data)):
        current_date = (datetime.datetime.strptime(str(data['Date'].values[i]), "%m/%d/%Y")).date()
        current_time = datetime.datetime.strptime(str(data['Time'].values[i]), "%H:%M:%S").time()
        current_datetime = datetime.datetime.combine(current_date,current_time)
        data['DateTime'][i] = datetime.datetime.strptime(str(current_datetime), "%Y-%m-%d %H:%M:%S")
        data['Date'][i] = data['DateTime'][i].date()
        data['Time'][i] = data['DateTime'][i].time()
except:
    pass


d = pd.DataFrame(data).groupby([data['Date']])
k_W = [9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
eta_W = [0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
k_A_W = [7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = [0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
eta_B_W = [0.0029*9.974510910778044]*len(data) #heat absorption coeff of bottom surface
k_hive = [1.5540477862285431]*len(data)
        
data['k_W'] = k_W
data['eta_W'] = eta_W
data['k_A_W'] = k_A_W
data['eta_A_W'] = eta_A_W
data['eta_B_W'] = eta_B_W
data['k_hive'] = k_hive

In [ ]:
h_max = 1
l = 96/h_max
params = Parameters()
a = 0
for i,j in d:
    params.add('h_'+str(a), value = 0.5, min = 0.1, max = h_max)
    try:
        params.add('theta_'+str(a), value = np.mean(j['Temp']), min = np.min(j['Temp']), max = np.max(j['Temp']))
    except:
        params.add('theta_'+str(a), value = np.mean(j['Temp']), min = np.min(j['Temp']) - 0.05, max = np.max(j['Temp']) + 0.05)
    a = a+1             
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)

result = minimize(residual, params, args=(t,data, l), method='leastsq',nan_policy='omit')
sig_min = data['Temp'].values + result.residual[:len(data['Temp'].values)].reshape(data['Temp'].shape)

In [ ]:
data['Recons_P_C_2'] = sig_min

h = np.array([])
theta = np.array([])
a = 0
for i in range(len(d)):
    h = np.concatenate((h, np.array([result.params['h_'+str(a)].value]*24)))
    theta = np.concatenate((theta, np.array([result.params['theta_'+str(a)]]*24)))
    a = a+1
data['h_P_C_2'] = h
data['theta_P_C_2'] = theta

In [ ]:
data.to_csv('C:\Bee Temp Data\C260_w_Env.csv')

In [ ]:
data = pd.read_csv('C:\Bee Temp Data\C260_w_Env.csv')
plt.figure(figsize = (10,4), dpi = 200)
ax1 = plt.subplot(211)

ax1.plot(data['Recons_P_C_2'], label = 'Reconstructed Core Temp w/o ice', color = 'blue')
ax1.plot(data['Recons_P_C_100'], label = 'Reconstructed Core Temp /w ice', color = 'red') 
ax1.plot(data['Temp'], label = 'Recorded Core Temperature /w ice', color = 'orange')
#ax1.plot(data['theta_P_C'])
ax1.set_ylabel('Temperature in celcius')
ax1.legend()

ax2 = plt.subplot(212)
ax2.plot(data['h_P_C_100'], label = 'h /w ice', color = 'blue')
ax2.plot(data['h_P_C_2'], label = 'h w/o ice', color = 'red') 
ax2.set_xlabel('Samples per hour')
ax2.set_ylabel('h')
ax2.set_ylim([0,1])
ax2.legend()
plt.show()

In [ ]:
plt.plot(data['h_P_C'], color='red')
plt.plot(data['h_P_C_2'], color='blue')
#plt.ylim([0,1])

In [ ]:
plt.plot(data['theta_P_C'], color = 'red')
plt.plot(data['theta_P_C_2'], color = 'blue')